# Segmentation Notebook

### <font color='red'> After clicking on a code cell, press "Shift+Enter" to run the code, or click on the "Run" button in the toolbar above.<br>

### Replace "..." signs with the appropriate path to your data.
</font>

In [ ]:
# from tapenade import get_path_to_demo_folder
from tapenade.preprocessing import (
    global_contrast_enhancement,
    local_contrast_enhancement,
    segment_stardist
)
from tapenade import get_path_to_demo_folder
from tapenade.preprocessing import change_array_pixelsize
from tapenade.preprocessing.segmentation_postprocessing import remove_small_objects, remove_labels_outside_of_mask
import numpy as np
import tifffile
import matplotlib.pyplot as plt
import skimage
from pathlib import Path
from skimage.measure import regionprops

Enter the path to the image you want to segment. The mask is optional, but it is recommended if you use the local contrast enhancement method (and do not load data that is already enhanced that way) or if you need to post-process the segmentation results (e.g to remove labels outside the mask or touching the borders).

In [ ]:
path_to_data = get_path_to_demo_folder()

data = tifffile.imread(path_to_data / "image_isotropized_enhanced.tif")
mask = tifffile.imread(path_to_data / "mask_isotropized.tif") # optional, read text above

## Quick note about pre-processing

The StarDist model we provide works best with roundish objects of approximately 15 pixels in diameters in all directions, and with images that have been normalized between 0 and 1.

If your image already has the appropriate size and is normalized between 0 and 1, you can skip this and directly go to the prediction.

To prepare your data, we recommend using the preprocessing notebook, particularly the following steps:
 - Adapting the image pixel size so that all objects are approximately 15 pixels in diameters in all directions using the `change_array_pixelsize` function. In our case, this meant resizing to the isotropic voxel size of (0.62, 0.62, 0.62) µm/pix. Do not hesitate to try different sizes on a subset of your data to optimize the result.
 - Normalizing the image values using `global_contrast_enhancement` or `local_contrast_enhancement`. Use the latter if your image is very deep, as it can improve performance in deeper planes by enhancing the contrast using local statistics.

For the purpose of this demo, the data we load has already been pre-processed, but in your case, you can go through the preprocessing notebook to prepare your data, or uncomment the following lines to run the pre-processing steps directly in this notebook if you are familiar with the functions already.

In [ ]:
# data_isotropized = change_array_pixelsize(
#     data,
#     input_pixelsize=..., # replace with the input pixelsize in ZYX order, e.g. (1, 0.5, 0.5)
#     output_pixelsize=(0.621, 0.621, 0.621), # isotropic pixelsize
#     order=1, # interpolation order, 1 for images, 0 for masks and labels
# )
# mask_isotropized = change_array_pixelsize(
#     mask,
#     input_pixelsize=..., # replace with the input pixelsize in ZYX order, e.g. (1, 0.5, 0.5)
#     output_pixelsize=(0.621, 0.621, 0.621), # isotropic pixelsize
#     order=0, # interpolation order, 1 for images, 0 for masks and labels
# )

# data_isotropized_normalized = local_contrast_enhancement(
#     data_isotropized,
#     mask=mask_isotropized,
#     box_size=10,
#     perc_low=1, perc_high=99
# )

data_isotropized_normalized = data # demo data is already preprocessed
mask_isotropized = mask

## Run the prediction using StarDist3D

We provide the function `segment_stardist`, which can be used to detect nuclei in 3D images using the StarDist model. The folder containing the weights and config is automatically downloaded in this demo, and can also be found [here](https://zenodo.org/records/14748083).

In [ ]:
path_stardist_model = Path(path_to_data / "tapenade_stardist") # folder containing weights

labels = segment_stardist(
    data_isotropized_normalized, # data already preprocessed
    path_stardist_model,
)

If you want to resize the labels back to the original pixel size, you can use the function `resize` from the package `skimage.transform`. 

In [ ]:
labels_at_array_pixelsize = skimage.transform.resize(
    labels,
    data.shape,
    anti_aliasing=False,
    order=0,
    preserve_range=True,
)

If you want to save the results, you can use the `tifffile.imwrite` function.

In [ ]:
tifffile.imwrite(..., labels) # replace ... with the path where you want to save the labels as a tif file

## Check results (napari required)

In [ ]:
try:
    import napari
    viewer=napari.Viewer()
    viewer.add_image(data_isotropized_normalized, colormap='inferno')
    viewer.add_labels(labels)
    napari.run()
except ImportError:
    print("Napari is not installed, skipping visualization.")

## Post-processing

### 1. Removing labels outside of mask (inside/outside of the tissue)

Due to the presence of noise in the image, the segmentation can sometimes produce labels that are not fully contained in the mask. We provide the function `remove_labels_outside_of_mask` to remove these labels. It takes as input the labels and the mask, and removes the labels that are not fully contained in the mask.

To create a mask of your image, use the preprocessing notebook or the napari plugin napari-tapenade-processing

In [ ]:
labels_filtered = remove_labels_outside_of_mask(
    mask=mask_isotropized, 
    labels=labels
)

### 2 - Filter small volumes in the segmentation.

First, plot the histogram of cell volumes to evaluate the threshold

In [ ]:
props=regionprops(labels)
volumes = np.array([prop.area for prop in props])

plt.hist(volumes, bins=100)
plt.title('Histogram of cell volumes')

Then, remove the objects smaller than ```size_min```

In [ ]:
#choose the size to filter
size_min = 400
labels_filtered = remove_small_objects(labels,size_min)

print('Before filtering :',len(np.unique(labels)),'labels \nAfter filtering :',len(np.unique(labels_filtered)),'labels')